# Day 10

In [13]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

enum class Direction(val shift: Position) {
    Up(Position(0, -1)),
    Left(Position(-1, 0)),
    Right(Position(1, 0)),
    Down(Position(0, 1))
}

enum class TileType(val symbol: Char, val directions: Set<Direction>) {
    VerticalPipe('|', setOf(Direction.Up, Direction.Down)),
    HorizontalPipe('-', setOf(Direction.Left, Direction.Right)),
    NorthEastBendPipe('L', setOf(Direction.Up, Direction.Right)),
    NorthWestBendPipe('J', setOf(Direction.Up, Direction.Left)),
    SouthWestBendPipe('7', setOf(Direction.Left, Direction.Down)),
    SouthEastBestPipe('F', setOf(Direction.Right, Direction.Down)),
    StartingPosition('S', Direction.entries.toSet()),
    Ground('.', emptySet()),
}

data class Position(
    val x: Int,
    val y: Int
) {
    operator fun plus(other: Position): Position =
        Position(x + other.x, y + other.y)
}

data class Tile(
    val type: TileType,
    val position: Position
)

val tilesFromSymbols = TileType.entries.associate { it.symbol to it }
fun Char.toTile(x: Int, y: Int): Tile =
    tilesFromSymbols[this]
        ?.let { type -> Tile(type = type, position = Position(x, y)) }
        ?: throw RuntimeException("No tile type associated with the symbol $this")

val lines = Path("./input").readLines().filter { it.isNotBlank() }
val tiles = lines.flatMapIndexed { y, line ->
    line.mapIndexed { x, char ->
        char.toTile(x, y)
    }
}


typealias TilesMap = Map<Position, Tile>

val map: TilesMap = tiles.associateBy { it.position }
val start = tiles.first { it.type == TileType.StartingPosition }

fun Tile.connectedPipes(map: TilesMap): Set<Tile> =
    type.directions
        .mapNotNull { map[position + it.shift] }
        .toSet()

tailrec fun findLoop(start: Tile, map: TilesMap, visitedNodes: List<Tile> = emptyList()): List<Tile> {
    val connections = start.connectedPipes(map).filter { it.connectedPipes(map).contains(start) }
    val availableConnections = connections - visitedNodes
    
    // Reached the full loop
    if (availableConnections.size == 0 && connections.any { it.type == TileType.StartingPosition }) {
        return visitedNodes + start
    }
    
    return findLoop(availableConnections.random(), map, visitedNodes + start)
}

// Part 1 
println(findLoop(start, map).size / 2)


6701
